In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

Loading checkpoint shards: 100%|██████████| 4/4 [00:48<00:00, 12.15s/it]


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('json', data_files={'train': 'json_example.json'})

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token 

# Tokenize the dataset
def tokenize_function(examples):
    inputs = examples["input"]
    outputs = examples["output"]

    model_inputs = tokenizer(inputs, truncation=True, padding=True)
    labels = tokenizer(outputs, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Adjust based on GPU memory
    num_train_epochs=3,  # Adjust epochs as needed
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)

# Fine-tune the model
trainer.train()

/Users/wrighte/repos/educational/honours/.venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RuntimeError: MPS backend out of memory (MPS allocated: 27.15 GB, other allocations: 384.00 KB, max allowed: 27.20 GB). Tried to allocate 64.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).